In [ ]:
import os
import sys
import matplotlib.pyplot as plt
current_dir = os.getcwd()
parent_parent_dir = os.path.abspath(os.path.join(current_dir, '../..')) # tweak so that you get dir of code project

sys.path.append(parent_parent_dir)


In [ ]:
import json
import pandas as pd
import ast
from sklearn.cluster import OPTICS
# %matplotlib qt5
%matplotlib inline

In [ ]:
from src.models.optics.get_clusters_from_optics_labels import get_clusters_from_optics_labels
from src.visualization.render_vehicle_track_cluster_in_notebook import render_vehicle_track_cluster_in_notebook
plt.rcParams.update({'font.size': 16})

def plot_optics_clusters(intersection_name, params_uid=None):

    print(f'NORMAL {intersection_name}')

    # get data
    data_path = f'{parent_parent_dir}/data/processed/{intersection_name}_cuid.csv'
    df_cuid = pd.read_csv(data_path)
    df_cuid_grouped_path = data_path.replace('.csv', '_grouped.csv')
    df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
    df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
    df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))


    def get_optics_params_from_metric(optics_params, metric):
        optics_params = optics_params[metric]
        dtw_dist_matrix_key = optics_params['dtw_key']
        max_eps = optics_params['epsilon']
        min_samples = optics_params['min_samples']
        cluster_method = optics_params['cluster_method']
        xi = optics_params['xi']
        kwargs = {
            'max_eps': max_eps,
            'min_samples': min_samples,
            'cluster_method': cluster_method,
            'xi': xi
        }
        filtered_kwargs = {k: v for k, v in kwargs.items() if v is not None}
        return dtw_dist_matrix_key, filtered_kwargs
    
    # check for params uid 
    if params_uid is not None:
        opt_params_path = f'{parent_parent_dir}/data/processed/{intersection_name}_optics_vehicle_paths_optimized_params_{params_uid}.json'
    else:
        opt_params_path = f'{parent_parent_dir}/data/processed/{intersection_name}_optics_vehicle_paths_optimized_params.json'
    with open(opt_params_path, 'r') as f:
        optics_params = json.load(f)

    # define optics params, dtw distance matrix key
    dtw_key_silhouette, kwargs_silhouette = get_optics_params_from_metric(optics_params, 'silhouette')
    print(f'dtw_key_silhouette: {dtw_key_silhouette}')
    print(f'kwargs_silhouette: {kwargs_silhouette}')
    dtw_key_davies_bouldin, kwargs_davies_bouldin = get_optics_params_from_metric(optics_params, 'davies_bouldin')
    dtw_key_calinski_harabasz, kwargs_calinski_harabasz = get_optics_params_from_metric(optics_params, 'calinski_harabasz')

    # import dtw distance matrix
    dtw_distance_matrix_path = f'{parent_parent_dir}/data/processed/{intersection_name}_diff_itakura_slope_dtw_matrices.json'
    with open(dtw_distance_matrix_path) as f:
        dtw_distance_matrix = json.load(f)

    # get the distance matrix
    dtw_dist_matrix_silhouette = dtw_distance_matrix[dtw_key_silhouette]
    dtw_dist_matrix_davies_bouldin = dtw_distance_matrix[dtw_key_davies_bouldin]
    dtw_dist_matrix_calinski_harabasz = dtw_distance_matrix[dtw_key_calinski_harabasz]

    # create optics models for each metric
    optics_silhouette = OPTICS(metric='precomputed', **kwargs_silhouette).fit(dtw_dist_matrix_silhouette)
    optics_davies_bouldin = OPTICS(metric='precomputed', **kwargs_davies_bouldin).fit(dtw_dist_matrix_davies_bouldin)
    optics_calinski_harabasz = OPTICS(metric='precomputed', **kwargs_calinski_harabasz).fit(dtw_dist_matrix_calinski_harabasz)

    # get labels for all models
    labels_silhouette = get_clusters_from_optics_labels(optics_silhouette.labels_)
    labels_davies_bouldin = get_clusters_from_optics_labels(optics_davies_bouldin.labels_)
    labels_calinski_harabasz = get_clusters_from_optics_labels(optics_calinski_harabasz.labels_)
    print(f'labels_silhouette: {labels_silhouette}')

    # plot models next to each other
    fig1, ax1 = plt.subplots(1, 1, figsize=(10,10))
    fig2, ax2 = plt.subplots(1, 1, figsize=(10,10))
    fig3, ax3 = plt.subplots(1, 1, figsize=(10,10))
    axs = [ax1, ax2, ax3]

    render_vehicle_track_cluster_in_notebook(axs[0], df_cuid, df_cuid_grouped, labels_silhouette)
    render_vehicle_track_cluster_in_notebook(axs[1], df_cuid, df_cuid_grouped, labels_davies_bouldin)
    render_vehicle_track_cluster_in_notebook(axs[2], df_cuid, df_cuid_grouped, labels_calinski_harabasz)
    fig1.canvas.manager.set_window_title(f'{intersection_name} - Silhouette')
    fig2.canvas.manager.set_window_title(f'{intersection_name} - Davies Bouldin')
    fig3.canvas.manager.set_window_title(f'{intersection_name} - Calinski Harabasz')

    print('********************************************')
    print(f'{intersection_name}')
    print('********************************************')

In [ ]:
from src.visualization.render_vehicle_track_cluster_in_notebook import render_vehicle_track_cluster_in_notebook
from src.models.optics.get_clusters_from_optics_labels import get_clusters_from_optics_labels


def plot_optics_clusters_acc_score(intersection_name, params_uid=None):

    print(f'ACC {intersection_name}')

    # get data
    data_path = f'{parent_parent_dir}/data/processed/{intersection_name}_cuid.csv'
    df_cuid = pd.read_csv(data_path)
    df_cuid_grouped_path = data_path.replace('.csv', '_grouped.csv')
    df_cuid_grouped = pd.read_csv(df_cuid_grouped_path)
    df_cuid_grouped['x'] = df_cuid_grouped['x'].apply(lambda x: ast.literal_eval(x))
    df_cuid_grouped['y'] = df_cuid_grouped['y'].apply(lambda y: ast.literal_eval(y))


    def get_optics_params(optics_params):
        dtw_dist_matrix_key = optics_params['dtw_key']
        max_eps = optics_params['epsilon']
        min_samples = optics_params['min_samples']
        cluster_method = optics_params['cluster_method']
        xi = optics_params['xi']
        kwargs = {
            'max_eps': max_eps,
            'min_samples': min_samples,
            'cluster_method': cluster_method,
            'xi': xi
        }
        filtered_kwargs = {k: v for k, v in kwargs.items() if v is not None}
        return dtw_dist_matrix_key, filtered_kwargs
    
    # check for params uid
    if params_uid is not None:
        opt_params_path = f'{parent_parent_dir}/data/processed/{intersection_name}_optics_vehicle_paths_optimized_params_acc_score_{params_uid}.json'
    else:
        opt_params_path = f'{parent_parent_dir}/data/processed/{intersection_name}_optics_vehicle_paths_optimized_params_acc_score.json'
    with open(opt_params_path, 'r') as f:
        optics_params = json.load(f)
    # import optimized parameters
    dtw_key_acc_score, kwargs_acc_score = get_optics_params(optics_params)

    # import dtw distance matrix
    dtw_distance_matrix_path = f'{parent_parent_dir}/data/processed/{intersection_name}_diff_itakura_slope_dtw_matrices.json'
    with open(dtw_distance_matrix_path) as f:
        dtw_distance_matrix = json.load(f)

    # get the distance matrix
    dtw_dist_matrix_acc_score = dtw_distance_matrix[dtw_key_acc_score]

    # create optics model
    optics_acc_score = OPTICS(metric='precomputed', **kwargs_acc_score).fit(dtw_dist_matrix_acc_score)

    # get labels for all models
    labels_acc_score = get_clusters_from_optics_labels(optics_acc_score.labels_)

    # plot models next to each other
    fig, axs = plt.subplots(1, 1, figsize=(10,10))
    fig.suptitle(f'Intersection: {intersection_name}')
    title_str = f'Acc Score: {optics_params["score"]}' + '\n' + f'Optimization init ID: {params_uid if params_uid is not None else "n/a"}'
    axs.set_title(title_str)
    render_vehicle_track_cluster_in_notebook(axs, df_cuid, df_cuid_grouped, labels_acc_score)

In [ ]:
%matplotlib inline  

for intersection_name in ['k729_2022']:
    plot_optics_clusters(intersection_name, params_uid='WDB')
plot_optics_clusters('k733_2018')
plot_optics_clusters('k733_2020')